In [4]:
import os
import requests
import lzma
import struct
import pandas as pd
from datetime import datetime, timedelta
from tqdm import tqdm

def download_dukas_minute_data(symbol, year, month, day, hour):
    # Dukascopy months are 0-indexed
    url = f"https://datafeed.dukascopy.com/datafeed/{symbol}/{year}/{month - 1:02d}/{day:02d}/{hour:02d}h_ticks.bi5"
    response = requests.get(url, stream=True)

    if response.status_code != 200:
        return None

    try:
        decompressed_data = lzma.decompress(response.content)
    except lzma.LZMAError:
        return None

    data = []
    for i in range(0, len(decompressed_data), 20):
        chunk = decompressed_data[i:i + 20]
        if len(chunk) != 20:
            continue
        try:
            millis, bid, ask, bid_vol, ask_vol = struct.unpack('>Qffhh', chunk)
            timestamp = datetime(year, month, day, hour) + timedelta(milliseconds=millis)
            data.append([timestamp, ask, bid, ask_vol, bid_vol])
        except (OverflowError, struct.error):
            continue

    if not data:
        return None

    return pd.DataFrame(data, columns=['time', 'ask', 'bid', 'ask_vol', 'bid_vol'])

# Example: download 1 full day of tick data
def download_full_day(symbol, year, month, day):
    all_data = []
    for hour in tqdm(range(24), desc=f"Fetching {year}-{month:02d}-{day:02d}"):
        df = download_dukas_minute_data(symbol, year, month, day, hour)
        if df is not None:
            all_data.append(df)
    return pd.concat(all_data) if all_data else pd.DataFrame()

# Example usage
df = download_full_day('XAUUSD', 2023, 1, 3)
df.to_csv('XAUUSD_ticks_2023_01_03.csv', index=False)


Fetching 2023-01-03: 100%|██████████| 24/24 [00:18<00:00,  1.29it/s]


In [6]:
from datetime import datetime
import dukascopy_python
from dukascopy_python import fetch, INTERVAL_MIN_1, OFFER_SIDE_BID, INTERVAL_TICK
from dukascopy_python.instruments import INSTRUMENT_FX_METALS_XAU_USD
import pandas as pd

# Parameters
instrument = INSTRUMENT_FX_METALS_XAU_USD  # XAU/USD instrument
start = datetime(2019, 12, 30)
end = datetime(2024, 1, 1)
interval = INTERVAL_MIN_1     # 1-minute candles
offer_side = OFFER_SIDE_BID    # or OFFER_SIDE_ASK

# Fetch data
df = fetch(
    instrument=instrument,
    interval=interval,
    offer_side=offer_side,
    start=start,
    end=end,
)

# Rename 'timestamp' column to 'datetime'
df = df.rename_axis('datetime').reset_index()

# Set 'datetime' as index
# df.set_index('datetime', inplace=True)


# Save to CSV
df.to_csv('train.csv', index=True)
print(df.head())


INFO:DUKASCRIPT:current timestamp :2020-01-30T03:22:00
INFO:DUKASCRIPT:current timestamp :2020-02-29T01:33:00
INFO:DUKASCRIPT:current timestamp :2020-03-31T19:08:00
INFO:DUKASCRIPT:current timestamp :2020-05-01T13:11:00
INFO:DUKASCRIPT:current timestamp :2020-06-02T11:12:00
INFO:DUKASCRIPT:current timestamp :2020-07-02T05:14:00
INFO:DUKASCRIPT:current timestamp :2020-08-01T02:21:00
INFO:DUKASCRIPT:current timestamp :2020-09-01T20:20:00
INFO:DUKASCRIPT:current timestamp :2020-10-01T18:23:00
INFO:DUKASCRIPT:current timestamp :2020-11-02T13:23:00
INFO:DUKASCRIPT:current timestamp :2020-12-02T14:45:00
INFO:DUKASCRIPT:current timestamp :2021-01-05T12:11:00
INFO:DUKASCRIPT:current timestamp :2021-02-04T10:11:00
INFO:DUKASCRIPT:current timestamp :2021-03-08T08:11:00
INFO:DUKASCRIPT:current timestamp :2021-04-08T00:13:00
INFO:DUKASCRIPT:current timestamp :2021-05-07T18:20:00
INFO:DUKASCRIPT:current timestamp :2021-06-08T16:23:00
INFO:DUKASCRIPT:current timestamp :2021-07-08T14:23:00
INFO:DUKAS

                   datetime      open      high       low     close   volume
0 2019-12-29 23:00:00+00:00  1511.458  1511.738  1511.288  1511.418  0.05372
1 2019-12-29 23:01:00+00:00  1511.478  1511.718  1511.278  1511.368  0.03187
2 2019-12-29 23:02:00+00:00  1511.368  1511.428  1511.198  1511.348  0.02042
3 2019-12-29 23:03:00+00:00  1511.348  1511.368  1511.178  1511.238  0.01251
4 2019-12-29 23:04:00+00:00  1511.258  1511.418  1511.218  1511.228  0.01836


In [ ]:
from datetime import datetime, timedelta
import dukascopy_python
from dukascopy_python import fetch, INTERVAL_TICK, OFFER_SIDE_BID
from dukascopy_python.instruments import INSTRUMENT_FX_METALS_XAU_USD
import pandas as pd
import os

# Parameters
instrument = INSTRUMENT_FX_METALS_XAU_USD
start_date = datetime(2019, 12, 30)
end_date = datetime(2024, 1, 1)
interval = INTERVAL_TICK
offer_side = OFFER_SIDE_BID
output_file = 'XAUUSD_tick_Train.csv'

# Check if file exists to write headers only once
file_exists = os.path.isfile(output_file)

# Daily loop
current_date = start_date
while current_date < end_date:
    next_date = current_date + timedelta(days=1)
    print(f"Fetching data for: {current_date.date()}")

    try:
        df = fetch(
            instrument=instrument,
            interval=interval,
            offer_side=offer_side,
            start=current_date,
            end=next_date,
        )
        if not df.empty:
            df.to_csv(output_file, mode='a', header=not file_exists, index=True)
            file_exists = True  # After the first write, ensure header=False for next writes
        else:
            print(f"No data for {current_date.date()}")

    except Exception as e:
        print(f"Error fetching data for {current_date.date()}: {e}")

    current_date = next_date


Fetching data for: 2019-12-30


INFO:DUKASCRIPT:current timestamp :2019-12-30T15:03:46.658000
INFO:DUKASCRIPT:current timestamp :2019-12-30T21:03:57.157000
INFO:DUKASCRIPT:current timestamp :2019-12-30T23:46:00.496000


Fetching data for: 2019-12-31


INFO:DUKASCRIPT:current timestamp :2019-12-31T10:49:12.677000
INFO:DUKASCRIPT:current timestamp :2019-12-31T19:00:58.149000
INFO:DUKASCRIPT:current timestamp :2019-12-31T21:48:48.868000


Fetching data for: 2020-01-01
Fetching data for: 2020-01-02


...

INFO:DUKASCRIPT:current timestamp :2020-01-23T07:33:26.887000
INFO:DUKASCRIPT:current timestamp :2020-01-23T13:52:56.368000
INFO:DUKASCRIPT:current timestamp :2020-01-23T19:41:50.309000
INFO:DUKASCRIPT:current timestamp :2020-01-23T21:55:15.017000
INFO:DUKASCRIPT:current timestamp :2020-01-23T23:47:28.257000


Fetching data for: 2020-01-24


INFO:DUKASCRIPT:current timestamp :2020-01-24T03:27:27.106000
INFO:DUKASCRIPT:current timestamp :2020-01-24T15:15:57.306000
INFO:DUKASCRIPT:current timestamp :2020-01-24T20:13:41.601000
INFO:DUKASCRIPT:current timestamp :2020-01-24T22:19:58.743000
INFO:DUKASCRIPT:current timestamp :2020-01-24T23:46:40.869000


Fetching data for: 2020-01-25


INFO:DUKASCRIPT:current timestamp :2020-01-25T02:12:14.171000
